In [ ]:
pip install scanpy anndata igraph louvain leidenalg

In [ ]:
pip install numba==0.56.0

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
from scipy.io import mmread
import numpy as np
import anndata as ad
import os
import igraph
import leidenalg
from collections import defaultdict
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()

sc.settings.set_figure_params(dpi=80)
%matplotlib inline

In [2]:
def readData(sampleID, type="", age="", path="../project-files/HTAN_HTAPP/"):
    '''this function is specific to the HTAPP dataset because it contains channel1/2'''
    files = os.listdir(path)
    m = 0
    f = 0
    b = 0
    for file in files:
        if sampleID in file:
            if "channel1" in file:
                # print(file)
                if m == 0:
                    if file.endswith("matrix.mtx.gz"):
                        m = 1
                        matrix = sc.read_mtx('{}/{}'.format(path, file)).transpose()
                if f == 0:
                    if file.endswith("features.tsv.gz"):
                        f = 1
                        features = pd.read_csv('{}/{}'.format(path, file), sep='\t', names=['ensembl', 'symbol', 'type'])
                if b == 0:
                    if file.endswith("barcodes.tsv.gz"):
                        b = 1
                        barcodes = pd.read_csv('{}/{}'.format(path, file), sep='\t', names=['barcode'])
    matrix.obs_names = barcodes['barcode']
    matrix.var_names = features['symbol']
    adata = ad.AnnData(matrix)
    adata.obs["patient_id"] = sampleID
    adata.obs["type"] = type
    adata.obs["age"] = age
    adata.var_names_make_unique()
    return(adata)


In [3]:
# read meta data and store patient information
metaDict = defaultdict(list)
metaFile = open("../project-files/HTAN_HTAPP_metadata_updated.csv", "r")
metaFile.readline()
sampleList = []
for line in metaFile.readlines():
    line = line.rstrip().split(",")
    if line[6] in ["Lobular carcinoma NOS", "Lobular adenocarcinoma"] and line[30] not in metaDict["lobular"]:
        metaDict["lobular"].append(line[30])
    if line[6] == "Ductal carcinoma NOS" and line[30] not in metaDict["ductal"]:
        metaDict["ductal"].append(line[30])
    if len(line[33]) > 0 and float(line[33]) <= 45.5 and line[30] not in metaDict["young"]:
        metaDict["young"].append(line[30])
    if len(line[33]) > 0 and float(line[33]) > 45.5 and line[30] not in metaDict["old"]:
        metaDict["old"].append(line[30])
    sampleList.append(line[30])
metaFile.close()
metaFile = open("../project-files/HTAN_HTAPP_metadata.csv", "r")
metaFile.readline()
metaFile.readline()
metaFile.readline()
for line in metaFile.readlines():
    if line.startswith("segmentation"):
        break
    line = line.rstrip().split(",")
    if line[6] in ["Lobular carcinoma NOS", "Lobular adenocarcinoma"] and line[40] not in metaDict["lobular"]:
        metaDict["lobular"].append(line[40])
    if line[6] == "Ductal carcinoma NOS" and line[40] not in metaDict["ductal"]:
        metaDict["ductal"].append(line[40])
    if len(line[42]) > 0 and float(line[42]) <= 45.5 and line[40] not in metaDict["young"]:
        metaDict["young"].append(line[40])
    if len(line[42]) > 0 and float(line[42]) > 45.5 and line[40] not in metaDict["old"]:
        metaDict["old"].append(line[40])
    sampleList.append(line[40])
    # break
metaFile.close()
# metaDict
sampleDict = {}
for sample in list(set(sampleList)):
    if sample in metaDict['lobular'] and sample in metaDict['young']:
        sampleDict[sample] = ['lobular', 'young']
    elif sample in metaDict['lobular'] and sample in metaDict['old']:
        sampleDict[sample] = ['lobular', 'old']
    elif sample in metaDict['ductal'] and sample in metaDict['young']:
        sampleDict[sample] = ['ductal', 'young']
    elif sample in metaDict['ductal'] and sample in metaDict['old']:
        sampleDict[sample] = ['ductal', 'old']
# sampleDict

In [5]:
conc_list = []
count = 0
for key in sampleDict:
    count += 1
    sample = "HTAPP-{}".format(key[5:])
    print("{}: {}".format(count, sample))
    conc_list.append(readData(sample, type=sampleDict[key][0], age=sampleDict[key][1]))

1: HTAPP-806


/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


2: HTAPP-589


/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


3: HTAPP-213


...

/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


37: HTAPP-887


/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


38: HTAPP-944


/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
# combined_data.write("HTAPP_combined.h5ad")
combined_data = sc.concat(conc_list, label="patient_id", keys=[adata.obs['patient_id'][0] for adata in conc_list])
combined_data.write("HTAPP_combined.h5ad")
combined_data

/tmp/ipykernel_12117/2116206593.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_data = sc.concat(conc_list, label="patient_id", keys=[adata.obs['patient_id'][0] for adata in conc_list])
/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 173399040 × 33538
    obs: 'patient_id', 'type', 'age'

In [7]:
mt_genes = [gene for gene in combined_data.var_names if gene.startswith('MT-')]
adata = combined_data[:, ~combined_data.var_names.isin(mt_genes)]
adata

View of AnnData object with n_obs × n_vars = 173399040 × 33525
    obs: 'patient_id', 'type', 'age'

In [15]:
adata

View of AnnData object with n_obs × n_vars = 173399040 × 33525
    obs: 'patient_id', 'type', 'age'

In [17]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
# sc.pl.highly_variable_genes(adata)
# adata = adata[:, adata.var.highly_variable]
adata

filtered out 173119310 cells that have less than 200 genes expressed


/opt/conda/lib/python3.11/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.

filtered out 2264 genes that are detected in less than 3 cells
normalizing counts per cell


/opt/conda/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


    finished (0:00:01)


AnnData object with n_obs × n_vars = 279730 × 31261
    obs: 'patient_id', 'type', 'age', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'

In [ ]:
adata.write("HTAPP_filtered.h5ad")